In [1]:
from src import NeuralTester, ExperimentConfig
import torch
from src.learner import PymooLearner
from src.learner.auxiliary_components import PYMOO_AGE_MOEA_DEFAULT_PARAMS
from src.criteria.image_comparison import CFrobeniusDistance
from src.criteria.objective_functions import NaiveConfidenceBalance
from models import load_stylegan
import wandb
from _enable_logging import enable_logging

## Example usage of the Neural Tester using a MOO
We use wandb to log progress in various steps!

In [2]:
wandb.login()
enable_logging()  # Setup logging to be able to view logging outputs in notebooks.

wandb: Currently logged in as: oliverweissl. Use `wandb login --relogin` to force relogin


In [3]:
# Define the configurations for our experiments.
conf = ExperimentConfig(
    samples_per_class=2,
    generations=150,
    mix_dim_range=(0, 8),
    predictor=torch.load("../models/wrn_mnist.pkl"),  # The System under test (SUT).
    generator=load_stylegan("../models/sg2_mnist.pkl"),  # The generator network.
    metrics=[CFrobeniusDistance(), NaiveConfidenceBalance()],
    classes=10,
)

In [4]:
learner_params = PYMOO_AGE_MOEA_DEFAULT_PARAMS
learner_params["n_var"] = conf.genome_size
learner = PymooLearner(**learner_params)  # The learner for search based optimization of candidates.

In [5]:
# Here we initialize the Tester object.
tester = NeuralTester(
    config=conf,
    learner=learner,
    device=torch.device("cuda"),
)

In [ ]:
# We start the testing procedure.
tester.test()

[2024-11-20 16:53:32,357 - root - INFO] Start testing. Number of classes: 10, iterations per class: 2, total iterations: 20



[2024-11-20 16:53:33,832 - root - INFO] Generate seed(s) for class: 0.
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... 